# Turn parquet to pickle file

In [5]:
import sys
sys.path.append("../topic_discovery")
import numpy as np

In [6]:
from pyspark import SparkContext

sc = SparkContext("local", "Simple App")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/29 11:55:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
from pyspark import SQLContext
sqlContext = SQLContext(sc)

/anaconda/lib/python3.7/site-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [8]:
df = sqlContext.read.parquet("hdfs://nr-hbase-c-hdfs.service.consul.taboolasyndication.com/newsroom-backend/insights/wikipedia/hebrew_page_text/titles")

df.printSchema()

root
 |-- title: string (nullable = true)



In [9]:
df.show(10, True)

+--------------+
|         title|
+--------------+
|חוקי פרנל-ארגו|
| אנסטסיוס השני|
|   מפרץ סרסוטה|
| מפרץ סאראסוטה|
|   קיטוב רוחבי|
|   נערה בהפרעה|
|       קאן-קאן|
|       קאן קאן|
|  ס"כ נס ציונה|
|     עצמי כוזב|
+--------------+
only showing top 10 rows



In [10]:
import pandas as pd

pandas_df = df.toPandas()
pandas_df

,title
0,חוקי פרנל-ארגו
1,אנסטסיוס השני
2,מפרץ סרסוטה
3,מפרץ סאראסוטה
4,קיטוב רוחבי
...,...
509798,קניון מרטין הבן
509799,קניון מרטין ג'וניור
509800,קיי. ג'יי. מרטין
509801,יהודה גרינוולד (רב)


In [11]:
pandas_df.to_pickle("/cache/hebrew_title.pkl")

In [17]:
df = pandas_df.replace('בירושלים' ,'ירושלים')
df

,title
0,חוקי פרנל-ארגו
1,אנסטסיוס השני
2,מפרץ סרסוטה
3,מפרץ סאראסוטה
4,קיטוב רוחבי
...,...
509798,קניון מרטין הבן
509799,קניון מרטין ג'וניור
509800,קיי. ג'יי. מרטין
509801,יהודה גרינוולד (רב)


In [18]:
df.to_pickle("/cache/hebrew_title.pkl")

In [4]:
#!hadoop fs -put /cache/hebrew_title.pkl hdfs://nr-hbase-c-hdfs.service.consul.taboolasyndication.com/newsroom-backend/insights/wikipedia/hebrew_page_text
!hadoop fs -ls hdfs://nr-hdfs.service.consul.taboolasyndication.com/newsroom-backend/insights/wikipedia/hebrew_page_text/

Found 4 items
drwxr-xr-x   - newsroom newsroom          0 2021-07-08 03:47 hdfs://nr-hdfs.service.consul.taboolasyndication.com/newsroom-backend/insights/wikipedia/hebrew_page_text/20210707
-rw-r--r--   3 newsroom newsroom   14039889 2021-07-15 05:51 hdfs://nr-hdfs.service.consul.taboolasyndication.com/newsroom-backend/insights/wikipedia/hebrew_page_text/hebrew_title.pkl
-rw-r--r--   3 newsroom newsroom   13016681 2021-10-28 20:13 hdfs://nr-hdfs.service.consul.taboolasyndication.com/newsroom-backend/insights/wikipedia/hebrew_page_text/hebrew_title.txt
-rw-r--r--   3 newsroom newsroom    4186865 2021-11-01 23:38 hdfs://nr-hdfs.service.consul.taboolasyndication.com/newsroom-backend/insights/wikipedia/hebrew_page_text/hebrew_title.txt.gz


In [2]:
!hadoop fs -get hdfs://nr-hdfs.service.consul.taboolasyndication.com/newsroom-backend/insights/wikipedia/hebrew_page_text/hebrew_title.pkl /cache

2022-09-05 06:56:44,456 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


# Load picked file and turn title list to set

In [5]:
import pandas as pd

unpickled_df = pd.read_pickle("/cache/hebrew_title.pkl")
unpickled_df

,title
0,חוקי פרנל-ארגו
1,אנסטסיוס השני
2,מפרץ סרסוטה
3,מפרץ סאראסוטה
4,קיטוב רוחבי
...,...
509798,קניון מרטין הבן
509799,קניון מרטין ג'וניור
509800,קיי. ג'יי. מרטין
509801,יהודה גרינוולד (רב)


In [12]:
!pip install hebrew-tokenizer

Looking in indexes: https://artifactory.taboolasyndication.com/artifactory/api/pypi/python-public/simple
Looking in links: https://artifactory.taboolasyndication.com/artifactory/generic/topic-disco/whl
  Created wheel for hebrew-tokenizer: filename=hebrew_tokenizer-2.3.0-cp37-none-any.whl size=13431 sha256=fb2efb6e263da2dd6ae0f10311c54549e98a7aa66f2f025ae391dacfe6db4337
  Stored in directory: /root/.cache/pip/wheels/a8/43/3c/29d12ef4a8ef0dbfd79a36a1394d81034c7fe8fdf4a46a6bc6
Successfully built hebrew-tokenizer
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
import hebrew_tokenizer as ht
hebrew_text = "וגם: ביטוחי מנהלים - זמן להוריד מהמדף, המשפטים הבהיר שחוק התקשורת לא יוכל לעבור במושב הנוכחי, העדלאידע משתקת את המשק והפתעה במשחקי הבייסבול"
tokens = ht.tokenize(hebrew_text)  # tokenize returns a generator!
for grp, token, token_num, (start_index, end_index) in tokens:
    print('{}, {}'.format(grp, token))

HEBREW, וגם
PUNCTUATION, :
HEBREW, ביטוחי
HEBREW, מנהלים
PUNCTUATION, -
HEBREW, זמן
HEBREW, להוריד
HEBREW, מהמדף
PUNCTUATION, ,
HEBREW, המשפטים
HEBREW, הבהיר
HEBREW, שחוק
HEBREW, התקשורת
HEBREW, לא
HEBREW, יוכל
HEBREW, לעבור
HEBREW, במושב
HEBREW, הנוכחי
PUNCTUATION, ,
HEBREW, העדלאידע
HEBREW, משתקת
HEBREW, את
HEBREW, המשק
HEBREW, והפתעה
HEBREW, במשחקי
HEBREW, הבייסבול


In [7]:
title_list = unpickled_df['title'].to_list()

In [8]:
print('מפעלי' in title_list)
print('מפעל' in title_list)

False
True


In [9]:
print('צפוי' in title_list)
print('צפו' in title_list)

False
True


In [10]:
print('הרוסי' in title_list)
print('הרוס' in title_list)

True
False


In [20]:
from os import path

print(path.exists("/cache/123.txt"))

False


In [21]:
title_list = set(unpickled_df["title"])

In [22]:
print('בירושלים' in title_list)
print('ירושלים' in title_list)

False
True


In [28]:
print('ביטוחי' in title_list)
print('ביטוח' in title_list)

False
True


In [11]:
print('פרופי' in title_list)
print('פרופ' in title_list)

False
False


In [251]:
print("title" in unpickled_df)

True


In [264]:
wikipedia_text_file = None
if not wikipedia_text_file is None:
    print("Not None")
else:
    print("None")

None


In [269]:
df = pd.DataFrame()
def get_title_list(df):
    if "title" not in df:
        print("Wikipedia data incorrect! Please check: {}".format(wikipedia_text_file))
        return set()
    return set(df["title"])

title_list = get_title_list(df)

Wikipedia data incorrect! Please check: None


In [274]:
print("צ'כיה" in title_list)

True


# Calling Embark to get Hebrew entities

In [179]:
from deep_common.deep_common_embark_connector import EmbarkConnector
from deep_common.deep_const import EmbarkChannelAddress, EMBARK_SCORING_BATCH_SIZE, EMBARK_TIMEOUT_DEEP_SCORER

embark_connector = EmbarkConnector("embark.taboolasyndication.com:5051")

# input format is a json string of build_predict_model that is defined in py36/taboola/nlu/nlu_api_server/api_server.py
#batch = ['{"input": "המוזיקה חוזרת לטברנות ביוון וטורקיה שוב זמינה: תמונת מצב ביעדים האהובים", "config": {"limit": 10}, "language": "he" }']
batch = ['{"input": "זוכת פרס סוקולוב: קרינה שטוטלנד, עורכת לאשה", "config": {"limit": 10}, "language": "he" }']
# batch = ['How did Coronavirus break out? Theories abound as researchers race to solve genetic detective']
# model_label is the model name that is defined in copy_model.sh
result = embark_connector.get_entity_embeddings(model_label='entities-multi',
                                                entity_ids=batch,
                                                batch_size=250,
                                                timeout_per_batch=30,
                                                sync=True)

In [232]:
print(result.translations)
entities = get_entites(result)
print(entities)

['{"IOB_tags": ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"], "entities": []}', '{"IOB_tags": ["B-Location", "O", "O", "B-Person", "O", "O", "O", "O", "O", "O"], "entities": [["\\u05d1\\u05e8\\u05d1\\u05d9\\u05d1\\u05d0\\u05d9", 0.9771886467933655, "Location"], ["\\u05e4\\u05e8\\u05e5", 0.9912943243980408, "Person"]]}', '{"IOB_tags": ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-Location"], "entities": [["\\u05e9\\u05db\\u05dd", 0.9338584542274475, "Location"]]}', '{"IOB_tags": ["O", "O", "O", "O", "B-Location", "O", "B-Organization", "O", "O", "O", "O"], "entities": [["\\u05d1\\u05d9\\u05e8\\u05d5\\u05e9\\u05dc\\u05d9\\u05dd", 0.9927451610565186, "Location"], ["\\u05e6\\u05d4\\u05dc", 0.8921674489974976, "Organization"]]}', '{"IOB_tags": ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"], "entities": []}', '{"IOB_tags": ["O", "O", "O", "O", "O", "O", "B-Location"], "entities": [["\\u05d1\\u05d7\\u05d3\\u05e8\\u05d4", 0.999

In [233]:
import json

def get_entites(result):
    entities = []
    for prediction in result.translations:
        jsonStr = json.loads(prediction)
        for key in jsonStr:
            if key == "entities":
                for entity in jsonStr[key]:
                    #print(entity)
                    new_entity = match_entity_with_wikititle(entity[0], title_list)
                    entities.append(new_entity)
    return entities

In [231]:
print(len(title_example))
for entity in entities:
    print(entity)

98
[]
['ברביבאי', 'פרץ']
['שכם']
['בירושלים', 'צהל']
[]
['חדרה']
['לופן']
['טאקו']
['שמחה רוטמן']
['איטליה', 'ונציה']
['תמא 38']
['אילת', 'מגרעין נחל']
[]
['גדעון סער', 'מנדלבליט']
['בנט', 'עמנואל מורנו', 'עמנואל']
['פיניקס']
[]
[]
['מיקי זוהר']
['להפועל תא', 'דמארי']
[]
['נתניהו ובנט', 'רהב', 'ישראל']
[]
[]
['מיקרוסופט']
['ליצמן']
['הורוביץ']
['לישראל בטוקיו', 'אנדי מורז']
[]
[]
['טודו בום', 'סטטיק', 'לירז רוסו', 'ושרית פולק']
['סקס אפיל']
['ארגנטינה', "צ'ילה"]
['מכבי תא', 'גלבוע']
['נתניהו']
['נגב', 'יהודי']
['שיק שוק', "צ'כיה", 'סקוטלנד']
['ישיבת נווה הרצוג']
['צבי טאו', 'סמוטריץ']
['רשות המיסים', 'תל אביב']
['פולקסווגן']
['ליברמן נפגש', 'ההסתדרות']
[]
['ניקול ראידמן', 'טופלס']
[]
['סכנין', 'באר שבע']
[]
['בנט']
[]
['בנט', 'חמאס']
['חרדית', 'ממשלת ההסתה']
[]
['עכו לטובת']
['אחינועם ניני', 'נתניהו']
["צ'אט", 'ספורט 5']
[]
['ממשלת בנט - לפיד', 'כנסת']
['שיק שוק', "צ'כיה", 'סקוטלנד']
['בירם', 'בני סכנין']
['צסקא ואנדולו', 'תאודוסיץ']
['ליברמן', 'הבסטיליה']
[]
['בינגו ביבי']
['הורוסקופ'

In [29]:
def match_entity_with_wikititle(entity, title_list):
    if entity in title_list:
        return entity
    elif entity[1:] in title_list:
        return entity[1:]
    else:
        return entity

In [43]:
def _match_entity_with_wikititle(entity, title_list):
    '''
    Match entity without prefix (first char) with wikipedia title. if there, we keep the removed prefix one, if not, we keep original entity.
    '''
    if entity in title_list:
        return entity
    elif entity[0] in ['ש', 'ל', 'מ', 'ב', 'ו'] and entity[1:] in title_list:
        return entity[1:]
    else:
        return entity

title_list = ['חוקי פרנל-ארגו', 'אנסטסיוס השני', 'צ‘כיה']
entity = 'חוקי פרנל-ארגו'

new_entity = _match_entity_with_wikititle(entity, title_list)
print(new_entity)

assert 'צ‘כיה' == new_entity

חוקי פרנל-ארגו


AssertionError: 

In [35]:
import taboola.common.datasethelper.hdfs_utils as hdfs_utils

hdfs_prefix = "hdfs://nr-hbase-c-hdfs.service.consul.taboolasyndication.com"
hdfs_sub_data_dir = "/newsroom-backend/jupyter/notebooks/allen.wu/200_hebrew_titles.json"
hdfs_data_dir = hdfs_prefix + hdfs_sub_data_dir
hdfs_utils.get(hdfs_data_dir, "/cache") 

OSError: Failed calling subprocess: hdfs dfs -get hdfs://nr-hbase-c-hdfs.service.consul.taboolasyndication.com/newsroom-backend/jupyter/notebooks/allen.wu/200_hebrew_titles.json /cache with error [b"get: `/cache/200_hebrew_titles.json': File exists"]

In [246]:
file = '/cache/200_hebrew_titles.json'

with open(file,'r') as f:
    data = json.loads(f.read())

title_example = []
for key in data:
    title = data[key]['title']
    title_input = get_input(title)
    batch = []
    batch.append(title_input)
    result = embark_connector.get_entity_embeddings(model_label='entities-multi',
                                    entity_ids=batch,
                                    batch_size=250,
                                    timeout_per_batch=30,
                                    sync=True)
    print(result.translations)
    if 'OOV' not in result.translations:
        entities = get_entites(result)
        data[key]['entities'] = entities

['{"IOB_tags": ["B-Miscellaneous", "I-Miscellaneous", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-Person", "O", "B-Organization", "I-Organization"], "entities": [["\\u05de\\u05e9\\u05d7\\u05e7\\u05d9 \\u05d4\\u05e9\\u05e3", 0.9859762787818909, "Miscellaneous"], ["\\u05dc\\u05d0\\u05e8\\u05d6", 0.8684145212173462, "Person"], ["\\u05e8\\u05e9\\u05ea 13", 0.9811676144599915, "Organization"]]}']
['{"IOB_tags": ["O", "O", "O", "B-Location", "B-Location", "I-Location", "O", "O", "O", "O", "O", "O"], "entities": [["\\u05d1\\u05d9\\u05d5\\u05d5\\u05df \\u05d5\\u05d8\\u05d5\\u05e8\\u05e7\\u05d9\\u05d4 \\u05e9\\u05d5\\u05d1", 0.9989286661148071, "Location"]]}']
['{"IOB_tags": ["O", "O", "O", "B-Person", "B-Person", "O", "O", "O", "O", "O", "B-Location"], "entities": [["\\u05d1\\u05e0\\u05d8 \\u05dc\\u05d1\\u05d9\\u05d9\\u05d3\\u05df", 0.924168586730957, "Person"], ["\\u05d9\\u05e9\\u05e8\\u05d0\\u05dc", 0.9922974109649658, "Location"]]}']
['{"IOB_tags": ["O", "O", "O", "O", "B-Person", "I-Pe

In [242]:
for key in data:
    print(data[key]['title'])
    print(data[key]['entities'])
    break
    
with open('/cache/hebrew_output.json','w') as f:
    json.dump(data, f)

משחקי השף, עונה 5, פרק 16: ההפתעה שחיכתה לארז | רשת 13
['משחקי השף', 'ארז', 'רשת 13']


In [244]:
!hadoop fs -rm -r -skipTrash hdfs://nr-hbase-c-hdfs.service.consul.taboolasyndication.com/newsroom-backend/jupyter/notebooks/allen.wu/hebrew_output.json
!hadoop fs -put /cache/hebrew_output.json hdfs://nr-hbase-c-hdfs.service.consul.taboolasyndication.com/newsroom-backend/jupyter/notebooks/allen.wu/

Deleted hdfs://nr-hbase-c-hdfs.service.consul.taboolasyndication.com/newsroom-backend/jupyter/notebooks/allen.wu/hebrew_output.json
2021-07-11 23:01:09,439 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


In [181]:
import json

def get_input(text):
    text = text.replace('"', '')
    template = "{\"input\": \""+text+"\", \"config\": {\"limit\": 10}, \"language\": \"he\" }"
    return template

In [220]:
batch = []
for i in range(13, len(title_example)):
    print(i, i+1)
    if (i+1) < len(title_example):
        for title in title_example[i:i+1]:
            y = get_input(title)
            batch.append(y)
            result = embark_connector.get_entity_embeddings(model_label='entities-multi',
                                                entity_ids=batch,
                                                batch_size=250,
                                                timeout_per_batch=30,
                                                sync=True)

13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22
22 23
23 24
24 25
25 26
26 27
27 28
28 29
29 30
30 31
31 32
32 33
33 34
34 35
35 36
36 37
37 38
38 39
39 40
40 41
41 42
42 43
43 44
44 45
45 46
46 47
47 48
48 49
49 50
50 51
51 52
52 53
53 54
54 55
55 56
56 57
57 58
58 59
59 60
60 61
61 62
62 63
63 64
64 65
65 66
66 67
67 68
68 69
69 70
70 71
71 72
72 73
73 74
74 75
75 76
76 77
77 78
78 79
79 80
80 81
81 82
82 83
83 84
84 85
85 86
86 87
87 88
88 89
89 90
90 91
91 92
92 93
93 94
94 95
95 96
96 97
97 98


In [221]:
for title in title_example:
    y = get_input(title)
    batch.append(y)

In [222]:
#batch = ["{'input': 'משחקי השף, עונה 5, פרק 16: ההפתעה שחיכתה לארז | רשת 13', 'config': {'limit': 10}, 'language': 'he'}"]
result = embark_connector.get_entity_embeddings(model_label='entities-multi',
                                                entity_ids=batch,
                                                batch_size=250,
                                                timeout_per_batch=30,
                                                sync=True)

In [226]:
for x in result.translations:
    print(x)
    break

{"IOB_tags": ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"], "entities": []}


In [41]:
!pip freeze | grep pandas

pandas==1.0.5
